In [1]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [639]:
# May take a few seconds to run since the dataset is large
df = pd.read_excel('stock_data_super_large.xlsx')
df_risk_free = pd.read_excel('10_year_yield_US.xls')
df_mkt = pd.read_excel('SPY_data_final.xls', sheet_name = 'SPY_data')


In [640]:
# We have around 4000 companies in the dataset, 188 columns 
df.head()

,Identifier (RIC),Company Name,Industry,ESG_FY0,ESG_FY1,ESG_FY2,ESG_FY3,ESG_FY4,ESG_FY5,ESG_FY6,...,MCAP_FQ22,MCAP_FQ23,MCAP_FQ24,MCAP_FQ25,MCAP_FQ26,MCAP_FQ27,MCAP_FQ28,MCAP_FQ29,MCAP_FQ30,MCAP_FQ31
0,B,Barnes Group Inc,Machinery,47.280059,39.716448,39.364964,33.635145,24.153344,25.935442,18.761357,...,2.758021e+09,2.551881e+09,2.183471e+09,1.778151e+09,1.880405e+09,1.942959e+09,1.971735e+09,2.136121e+09,2.218038e+09,2.015078e+09
1,ITT.N,ITT Inc,Machinery,78.612516,70.722850,68.409017,59.747878,58.301170,54.710253,58.075415,...,3.637016e+09,3.401874e+09,3.211264e+09,2.881398e+09,3.313802e+09,3.250640e+09,2.988642e+09,3.732128e+09,3.607767e+09,3.706136e+09
2,GTLS.N,Chart Industries Inc,Machinery,56.790911,43.968150,41.893694,16.687256,21.707709,24.478215,NaN,...,1.073148e+09,1.102080e+09,1.004369e+09,7.381859e+08,6.643657e+08,5.485793e+08,5.867633e+08,1.091904e+09,1.071162e+09,1.042462e+09
3,PKOH.OQ,Park Ohio Holdings Corp,Machinery,11.351879,12.015005,12.092304,12.603972,12.932632,2.580297,NaN,...,4.514671e+08,5.277162e+08,4.575630e+08,3.473865e+08,5.381970e+08,4.654001e+08,3.641802e+08,6.055063e+08,6.581418e+08,7.885708e+08
4,ZWS.N,Zurn Elkay Water Solutions Corp,Building Products,61.015179,59.600678,42.460351,22.930730,21.765882,22.323763,NaN,...,2.371729e+09,2.013093e+09,2.199966e+09,1.996711e+09,2.049765e+09,1.818622e+09,1.703909e+09,2.434572e+09,2.714624e+09,2.868755e+09


In [641]:
# find the average market cap for each company, column lables MCAP_FY0 ~ MCAP_FY7
df['avg_mcap'] = df.loc[:,'MCAP_FQ0':'MCAP_FQ31'].mean(axis=1)
# same for ESG score
df['avg_esg'] = df.loc[:,'ESG_FY0':'ESG_FY6'].mean(axis=1)
# same for price
df['avg_price'] = df.loc[:,'Price_CM0':'Price_CM96'].mean(axis=1)
# book to market ratio
df['avg_btm'] = df.loc[:,'BTM_FQ0':'BTM_FQ31'].mean(axis=1)

In [642]:
# produce two dataframes, one with market cap lower than the median, one with market cap higher than the median
df_mcap_low = df[df['avg_mcap'] < df['avg_mcap'].median()]
df_mcap_high = df[df['avg_mcap'] >= df['avg_mcap'].median()]

# produce two dataframes, one with Price / Book Value Per Share lower than the median, one with Price / Book Value Per Share higher than the median
df_BM_low = df[df['avg_btm'] < df['avg_btm'].median()]
df_BM_high = df[df['avg_btm'] >= df['avg_btm'].median()]

# now for ESG score
df_esg_low = df[df['avg_esg'] < df['avg_esg'].median()]
df_esg_high = df[df['avg_esg'] >= df['avg_esg'].median()]

In [643]:
# check how many rows are in each dataframe
print(df_mcap_low.shape)

(2105, 219)


In [644]:
# find the return of each company for each CM period
# take in one dataframe, two integers, and two strings
def get_return(df, start, end, high_low, feature, freq):
    '''
    @param
    df: dataframe
    start: start period
    end: end period
    high_low: 'high' or 'low'
    feature: Default 'Price'
    freq: 'CM' or 'FY' or 'FQ'
    @return
    a dataframe with the return of each company for each CM period
    '''
    for i in range(start+1, end+1):
        df[feature+"_"+'Return_'+ freq + str(i)+"_"+high_low] = (df[feature + "_" + freq + str(i)] - df[feature + "_" + freq + str(i-1)]) / df[feature + "_" + freq  + str(i-1)]
    return df

In [645]:
# Market cap:
df_mcap_low_return = get_return(df_mcap_low, 0, 96, 'low', 'Price', 'CM')
df_mcap_high_return = get_return(df_mcap_high, 0, 96, 'high', 'Price', 'CM')

# ESG score:
df_esg_low_return = get_return(df_esg_low, 0, 96, 'low', 'Price', 'CM')
df_esg_high_return = get_return(df_esg_high, 0, 96, 'high', 'Price', 'CM')

# Price / Book Value Per Share:
df_BM_low_return = get_return(df_BM_low, 0, 96, 'low', 'Price', 'CM')
df_BM_high_return = get_return(df_BM_high, 0, 96, 'high', 'Price', 'CM')

In [646]:
# function to only include the last 96 columns (price data) and the second and third columns (company name and industry)
def get_price_data(df):
    '''
    @param
    df: dataframe
    @return
    a dataframe with only the price data and company name and industry, with NaNs dropped
    '''
    return df.iloc[:, [1, 2] + list(range(-96, 0))].dropna()

In [647]:
# market cap
df_mcap_low_dataset = get_price_data(df_mcap_low_return)
df_mcap_high_dataset = get_price_data(df_mcap_high_return)

# ESG score
df_esg_low_dataset = get_price_data(df_esg_low_return)
df_esg_high_dataset = get_price_data(df_esg_high_return)

# Price / Book Value Per Share
df_BM_low_dataset = get_price_data(df_BM_low_return)
df_BM_high_dataset = get_price_data(df_BM_high_return)


In [648]:
# this function find the average return for each CM period for each group
def get_return_avg(df, high_low):
    '''
    @param
    df: dataframe
    high_low: 'high' or 'low'
    @return
    a dataframe with the average return for each CM period for each group
    '''
    return df.loc[:,'Price_Return_CM1_'+high_low:'Price_Return_CM96_'+high_low].mean(axis=0).to_frame().T

def get_return_spread(df_low, df_high):
    '''
    @param
    df_low: dataframe of the low group
    df_high: dataframe of the high group
    @return
    a dataframe of the difference in average returns between the high and low groups
    '''
    # for each CM period, find the difference in average returns between the high and low groups (COLUMNS) and put them in a column with as SMB_1, SMB_2, etc.
    df_spread = pd.DataFrame()
    for i in range(0, 96):
        df_spread['CM'+str(i+1)] = df_low.iloc[:,i] - df_high.iloc[:,i]
    return df_spread
    


In [650]:
# calculate the difference in returns between each high and low group
# market cap
df_mcap_low_return_avg = get_return_avg(df_mcap_low_dataset, 'low')
df_mcap_high_return_avg = get_return_avg(df_mcap_high_dataset, 'high')
df_SMB_spread_mcap = get_return_spread(df_mcap_low_return_avg, df_mcap_high_return_avg).T.rename(columns={0: 'SMB'})

# ESG score
df_esg_low_return_avg = get_return_avg(df_esg_low_dataset, 'low')
df_esg_high_return_avg = get_return_avg(df_esg_high_dataset, 'high')
df_ESG_spread_esg = get_return_spread(df_esg_low_return_avg, df_esg_high_return_avg).T.rename(columns={0: 'ESG'})

# Price / Book Value Per Share
df_BM_low_return_avg = get_return_avg(df_BM_low_dataset, 'low')
df_BM_high_return_avg = get_return_avg(df_BM_high_dataset, 'high')
df_HML_spread_BM = get_return_spread(df_BM_low_return_avg, df_BM_high_return_avg).T.rename(columns={0: 'HML'})

In [651]:
# Now for Market return and Risk free rate
# calculate returns for both the market and risk free rate
df_risk_free_return = get_price_data(get_return(df_risk_free, 0, 96, 'na', 'Rf', 'CM')).T
df_mkt_return = get_price_data(get_return(df_mkt, 0, 96, 'na', 'Rm', 'CM')).T

# only include the last 96 rows (price data)
df_risk_free_return = df_risk_free_return.iloc[-96:].T
df_mkt_return = df_mkt_return.iloc[-96:].T

# find the get_return_spread
df_mkt_spread = get_return_spread(df_mkt_return, df_risk_free_return).T.rename(columns={0: 'Rm-Rf'})

In [654]:
# merge the three spreads into one dataframe
df_spread = pd.concat([df_mkt_spread, df_SMB_spread_mcap, df_HML_spread_BM, df_ESG_spread_esg], axis=1)

In [656]:
df_spread
# export to excel
#df_spread.to_excel('fama-french.xlsx')